In [ ]:
import os
from dotenv import load_dotenv
import sqlalchemy
import pymysql
import pandas as pd
import yfinance as yf
pymysql.install_as_MySQLdb()

In [ ]:
class LoadStockLst:
    def __init__(self, name, tkers):
        self.name = name
        self.tkers = tkers
    
    def getDbConn(self):
        load_dotenv()
        DB_ACCESS_KEY = os.getenv("DB_ACCESS_KEY")
        engine =\
            sqlalchemy.create_engine(DB_ACCESS_KEY+'/' \
                      +self.name)
        return engine.connect()

    def createStockLstSchema(self):
        try:
            load_dotenv()
            DB_ACCESS_KEY = os.getenv("DB_ACCESS_KEY")       
            engine = sqlalchemy.create_engine(DB_ACCESS_KEY)
            engine.execute(sqlalchemy.schema.CreateSchema(self.name))
        except Exception as e:
            pass
            
    def loadStockLstTables(self):
        goodTkers = [x for x in self.tkers if x == x] #drop all nan
        self.conn = self.getDbConn()
        for symbol in goodTkers:
            space_pos = symbol.find(' ')
            if space_pos != -1: # drop char after space in some case
                symbol = symbol[:space_pos]
            print(symbol)
            try:
                df = yf.download(symbol, start='2020-01-01')
                if len(df)>0:
                    df = df.reset_index()
                    try:
                        df.to_sql(symbol, self.conn, if_exists='replace')
                    except Exception as e:
                        print(e)
            except Exception as e:
                pass
            
    def procStockLst(self):
        print(self.tkers)
        self.createStockLstSchema() # create schema if not exists
        self.loadStockLstTables() #load tables

In [ ]:
cmy1Lst = ['BAC', 'C', 'WFC', 'JPM', 'V']
stkLst = LoadStockLst('CMY1', cmy1Lst)
stkLst.procStockLst()

### Create schema

In [ ]:
# arkxEtfLst=['ARKF']
# for etf in arkxEtfLst:
#     arkx= loadArkxEtf(etf)
#     arkx.procEtfx()

### Get symbols

In [ ]:
# arkk = ArkxEtf('ARKK')
# arkkTks = arkk.get_tickerlist()
# arkkTks = [x for x in arkkTks if x == x] #drop all nan
# print(arkkTks)

### Create tables